# Encuesta Nacional de Nutrición y Salud (ENNyS2)

## Introducción

La Segunda Encuesta Nacional de Nutrición y Salud (ENNyS 2) es un relevamiento realizado en Argentina por la Secretaría de Gobierno de Salud de la Nación (hoy Ministerio de Salud) entre octubre de 2018 y febrero de 2019.

Su objetivo principal es describir la situación alimentaria y de salud de la población, recolectando información detallada sobre:

* Consumo de alimentos y bebidas (recordatorio de 24 horas, frecuencia y tipo de alimentos).

* Estado nutricional (mediciones antropométricas y datos de composición corporal).

* Enfermedades crónicas y factores de riesgo (hipertensión, diabetes, colesterol alto, etc.).

* Condiciones sociodemográficas y del hogar (cobertura de salud, nivel educativo, ingresos, etc.).

* Hábitos de vida (actividad física, lactancia, consumo de tabaco y alcohol, etc.).

La encuesta utiliza un diseño probabilístico, estratificado y multietápico, lo que permite que los resultados sean representativos a nivel nacional y por regiones.

### Se estructura en cuestionarios específicos por grupo etario:

* C1: Niños/as menores de 5 años.

* C2: Niños/as y adolescentes de 5 a 12 años.

* C3: Adolescentes de 13 a 17 años.

* C4: Adultos de 18 años o más.

En este trabajo utilizaremos la base principal de publicada en el portal de datos abiertos del Ministerio de Salud, junto con las bases complementarias de alimentos y nutrientes, para realizar un análisis que vincule alimentación, estado de salud y factores asociados.

## Objetivos de la notebook

Explorar la base principal de la ENNyS2 (encuesta) para:
- ubicar variables clave (edad, sexo, indicadores de salud, educación, región),
- revisar cobertura de datos en adultos (≥18),
- generar un subset mínimo limpio para notebooks posteriores.
- Mostramos capacidades de la base sin plantear hipótesis temáticas.

## Estructura de las columnas en ENNyS2

La base principal de ENNyS2 contiene aproximadamente 1830 columnas.
Estas variables se organizan en módulos y prefijos que indican el tipo de información, la población objetivo y, en algunos casos, la posición del miembro del hogar.

### Convenciones de nombres:

Cn_: Variables de cuestionarios específicos según grupo etario:

C1_: Niños/as menores de 2 años (respondido por adulto responsable).

C2_: Niños/as y adolescentes de 2 a 12 años.

C3_: Adolescentes de 13 a 17 años.

C4_: Adultos de 18 años o más.

T_Cn_: Variables totales o derivadas de un módulo.
Ejemplo: T_C4_ActFisTotal (total de minutos de actividad física en adultos).

I_Dx_Cn_: Indicadores por día dentro del módulo.
Ejemplo: I_D1_C4_Kcal (ingesta calórica del día 1 en adultos).

Mn_: Información de miembros del hogar (hasta 15 posibles).
Ejemplo: M1_Edad, M2_Sexo (edad y sexo del miembro 1 y 2).

---

## 1. Cargar la base CSV

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA = Path("../data") # Previamente hay que descargar los datos manualmente y descomprimirlos en este directorio, mas detalles en el README.md
FILE = DATA / "ENNyS2_encuesta.csv"

df = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False) # Encoding para acentos

df.shape

(21358, 1830)

## Aclaración de como se realiza la encuesta

### 1. Formulario del hogar y de cada individuo

* Todos los hogares seleccionados responden un formulario sociodemográfico.

* En ese formulario se registra a cada integrante del hogar (edad, sexo, parentesco, variables socioeconómicas, etc.).

* Esa información forma parte de lo que llaman microdatos básicos (hogar e individuo).

### 2. Sorteo del individuo

* De cada hogar se sortea un integrante al azar.

* Ese integrante es el que va a responder el formulario individual específico según la edad.

    - 0 a 23 meses: Cuestionario C1

    - 2 a 12 años: Cuestionario C2

    - 13 a 17 años: Cuestionario C3

    - 18 años y más: Cuestionario C4

**IMPORTANTE**: Solo ese sorteado tiene la entrevista completa sobre salud, alimentación, antecedentes, etc.

### 3. Encuesta de recordatorio de 24 horas (R24H)

El mismo individuo sorteado es el que también responde el Recordatorio de 24 horas (qué comió y bebió el día anterior). Es decir: el R24H no se aplica a todos los integrantes del hogar, sino únicamente al seleccionado.

### 4. Quiénes quedan “no sorteados”

Los demás integrantes del hogar (los que no fueron seleccionados) solo tienen los datos básicos registrados en el cuestionario sociodemográfico.

Esos “no sorteados” no tienen:

cuestionario individual C1–C4,

ni recordatorio de 24 horas.


### 5. El R24H

El recordatorio de 24 horas (R24H) es una entrevista detallada sobre TODO lo que consumió la persona el día anterior (alimentos y bebidas, cantidad, horario, contexto).Se aplica solo al individuo sorteado del hogar.

* En la mayoría de los casos se aplica una sola vez (el día de la entrevista).

* A una submuestra se le vuelve a hacer un segundo recordatorio en otro día de la semana para capturar variabilidad intraindividual (no todos los días comemos igual).

* Qué pasa si no se puede realizar:

    - Si la persona no puede responder (ej: bebé de 0–23 meses), se entrevista al adulto responsable de la alimentación.

    - Si la entrevista no se logra completar, queda como registro perdido (faltante).

* En qué base está:

    - El R24H no se ve en las bases de hogar o cuestionarios C1–C4, sino en las bases específicas de alimentos y nutrientes:

    - Base de alimentos → cada fila = 1 alimento consumido.

    - Base de nutrientes → a partir de los alimentos, se calcula el desglose en energía, proteínas, vitaminas, minerales, etc.


## 2. Separamos en 4 subbases (C1 a C4)

La encuesta se realiza con cuatros formularios (C1 a C4) según la edad del encuestado.

In [25]:
print(df["E_CUEST"].unique())

df_c1 = df[df["E_CUEST"]=="0 a 23 meses"]
df_c2 = df[df["E_CUEST"]=="2 a 12 años"]
df_c3 = df[df["E_CUEST"]=="13 a 17 años"]
df_c4 = df[df["E_CUEST"]=="18 o mas años"]

print("0 a 23 meses:", df_c1.shape[0])
print("2 a 12 años:", df_c2.shape[0])
print("13 a 17 años:", df_c3.shape[0])
print("18 años y más:", df_c4.shape[0])


['18 o mas años' '2 a 12 años' '13 a 17 años' '0 a 23 meses']
0 a 23 meses: 5763
2 a 12 años: 5829
13 a 17 años: 2399
18 años y más: 7367


## ¿por qué los conteos de C1–C4 suman igual al total?

* En teoría del diseño: solo 1 integrante por hogar debía recibir C1–C4 y R24H.

* En la práctica de los microdatos todos los individuos del hogar aparecen con un campo “E_CUEST” asignado.

* Pero eso no significa que todos lo hayan respondido.

* El que fue sorteado tiene efectivamente todas las variables respondidas en su bloque C1, C2, C3 o C4.

* Los demás aparecen con el mismo layout de variables, pero vacías (celdas en blanco o con NA).


---

## 2. Seleccionamos columnas relevantes

In [4]:
# === 2. Columnas relevantes ===
columnas_relevantes = [
    # Datos básicos
    "Edadd", "C4_SEXO", "IMC", "Cobertura_salud", "IngHog_UC_QUINT_imp", "region",
    
    # Enfermedades crónicas
    "T_C4_SG_1_2_1",  # Diabetes
    "T_C4_SG_1_2_2",  # Enfermedad renal
    "T_C4_SG_1_2_3",  # Hipertensión o presión alta
    "T_C4_SG_1_2_4",  # Dislipemia, colesterol o triglicéridos altos
    "T_C4_SG_1_2_5",  # Anemia
    "T_C4_SG_1_2_6",  # Cardiopatía
    "T_C4_SG_1_2_7",  # Sonda vesical
    "T_C4_SG_1_2_8",  # Transplante de riñón
    "T_C4_SG_1_2_9",  # Obesidad
    "T_C4_SG_1_2_10", # EPOC
    "T_C4_SG_1_2_11", # Infarto
    "T_C4_SG_1_2_12", # Cáncer
    "T_C4_SG_1_2_13", # Leucemia
    "T_C4_SG_1_2_14"  # Otras enfermedades de larga duración
]

# === 3. Renombrar columnas ===
nombres_amigables = {
    "Edadd": "edad",
    "C4_SEXO": "sexo",
    "IMC": "imc",
    "Cobertura_salud": "cobertura_salud",
    "IngHog_UC_QUINT_imp": "quintil_ingresos",
    "region": "region",
    "T_C4_SG_1_2_1": "diabetes",
    "T_C4_SG_1_2_2": "enf_renal",
    "T_C4_SG_1_2_3": "hipertension",
    "T_C4_SG_1_2_4": "dislipemia",
    "T_C4_SG_1_2_5": "anemia",
    "T_C4_SG_1_2_6": "cardiopatia",
    "T_C4_SG_1_2_7": "sonda_vesical",
    "T_C4_SG_1_2_8": "transplante_renal",
    "T_C4_SG_1_2_9": "obesidad",
    "T_C4_SG_1_2_10": "epoc",
    "T_C4_SG_1_2_11": "infarto",
    "T_C4_SG_1_2_12": "cancer",
    "T_C4_SG_1_2_13": "leucemia",
    "T_C4_SG_1_2_14": "otras_enf_larga_duracion"
}

df_sel = df_ad[columnas_relevantes].rename(columns=nombres_amigables)

df_sel.head()


,edad,sexo,imc,cobertura_salud,quintil_ingresos,region,diabetes,enf_renal,hipertension,dislipemia,anemia,cardiopatia,sonda_vesical,transplante_renal,obesidad,epoc,infarto,cancer,leucemia,otras_enf_larga_duracion
976,1308,,,Obra social o prepaga,5,GBA,,,,,,,,,,,,,,
977,5737,,31.45034,Obra social o prepaga,4,GBA,,,,,,,,,,,,,,
980,5264,,17.09889,Obra social o prepaga,3,GBA,,,,,,,,,,,,,,
981,672,,18.91265,Obra social o prepaga,3,GBA,,,,,,,,,,,,,,
983,2100,,19.14394,Solo pública,4,GBA,,,,,,,,,,,,,,
